## 當前進度說明

### html上傳 -> 去背、Color、Pattern、Type -> 寫入MySQL -> 回傳成功/失敗
### html選擇時間地點 -> 爬取天氣資料 -> 回傳均溫/無法查詢 ->>>>>>> 
### html 點則「我的衣櫃」 -> SHOW 資料庫內所有衣服

## 導入套件

In [1]:
import tensorflow as tf
import pandas as pd
from flask import Flask, request, render_template, jsonify
import os
import cv2
import numpy as np
import colorList2
from rembg import remove
import mysql.connector
import requests
import re
import time
from datetime import date
from datetime import timedelta
import random
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from pandas.core.frame import DataFrame
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import base64
import pymysql
import matplotlib.pyplot as plt
from flask import Flask, render_template, request, send_file
import io
from io import BytesIO
from PIL import Image
from PIL import ImageTk
import json

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## 導入模型

In [2]:
from keras.models import load_model
model = load_model('finalmodel')
#model = load_model('finalmodel2.0')

2023-12-05 14:32:47.750136: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-05 14:32:47.750158: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-05 14:32:47.750161: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-05 14:32:47.750342: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-05 14:32:47.750360: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 開始執行：正式版本（0張圖片重新抓）

In [3]:
app = Flask(__name__)

# 設置上傳圖片的配置
app.config['UPLOAD_FOLDER'] = 'uploads'
if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
#設置圖片大小
img_height=28
img_width=28
batch_size=6

# 連接數據庫
#db = mysql.connector.connect(host="127.0.0.1", user="root", passwd="900603betty", database="ALL_CLOTHES_SQL", connection_timeout=6000)
#cursor = db.cursor()

url="https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-D0047-091?Authorization=rdec-key-123-45678-011121314"
res=requests.get(url)
resjson=json.loads(res.text)

data=[]

for i in range(0, 22):
    location=resjson['records']['locations'][0]['location'][i]['locationName']
    aa=resjson['records']['locations'][0]['location'][i]['weatherElement'][1]['time']
    
    for j in aa:
        for o in j['elementValue']:
            if '12:00:00' in j["startTime"] or '06:00:00' in j["startTime"]:
                da=j["startTime"].split(" ")
                date=da[0]
                temperature=o['value'] + "°C"
                data.append([location,date,temperature])

df =pd.DataFrame(data,columns=['Location','Date','Temperature'])

df['Date'] = df['Date'].str.replace('-', '/')
#df['Location'] =df['Location'].str.replace('臺北', '台北')

df['Date'] = df['Date'].astype("string")
df['Location'] = df['Location'].astype("string")
df['Temperature'] = df['Temperature'].astype("string")

def get_color(hsv):
    maxsum = -100
    color = None
    color_num = 0
    color_dict = colorList2.getColorList()
    for d in color_dict:
        mask = cv2.inRange(hsv,color_dict[d][0],color_dict[d][1])
        binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]
        binary = cv2.dilate(binary,None,iterations=2)
        cnts, hiera = cv2.findContours(binary.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        sum = 0
        for c in cnts:
            sum+=cv2.contourArea(c)
        if sum > maxsum :
            maxsum = sum
            color = d
        if sum > 2500:
            color_num +=1
    
    pattern = None
    if color_num <= 3:
        pattern = 'n'
    else:
        pattern = 'y'
        
        
                 
    return color,pattern


def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
def get_temperature(location, dir_date):    
    temp = df[(df['Location'] == location) & (df['Date'] == dir_date)]
    
    if not temp.empty:
        temperature = temp['Temperature'].str.replace('°C', '').astype(int).values
        return temperature[0] if temperature.any() else None
    else:
        return None

    
    
 
          
        


@app.route('/')
def index():
    return render_template('index.html') 

@app.route('/about_page')
def about():
    return render_template('about.html') 
    
@app.route('/upload_page')
def upload_page():
    return render_template('html_cnn_py.html')  # 導向到上傳圖片的HTML頁面    


@app.route('/upload', methods=['POST'])
def upload_files():
    uploaded_files = request.files.getlist("my_image")  # "my_image" 是 HTML 表單字段的名稱
    file_paths = []

    for uploaded_file in uploaded_files:
        if uploaded_file and allowed_file(uploaded_file.filename):
            # 將上傳的圖片保存到伺服器的 'uploads' 目錄中
            filename = os.path.join(app.config['UPLOAD_FOLDER'], uploaded_file.filename)
            uploaded_file.save(filename)
            file_paths.append(filename)

            # 使用OpenCV讀取圖片，然後處理
            img_o = cv2.imread(filename)
            frame = remove(img_o)
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            print()
            print("---------------------------------------")
            print(uploaded_file.filename)
            clothes_info = get_color(hsv)
            clothes_color = clothes_info[0]
            clothes_pattern = clothes_info[1]
    
            type_list=['短褲','羽絨','外套','無洋','帽踢','長袖','背心','長洋','短袖','高領','長裙','長褲']
            temp_filename = 'temp_image.png'
            cv2.imwrite(temp_filename, frame)

            # 使用 tf.keras 加載圖片
            precl=[]
            prel=[]
            img_height=28
            img_width=28
            batch_size=6
            img = tf.keras.preprocessing.image.load_img(temp_filename, target_size=(img_height, img_width))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            predictions = model.predict(tf.expand_dims(img_array, 0)) #use mod as model name if use pickle
    
            iname=np.argmax(predictions)
            cname=type_list[iname] 
    
            prel.append(iname)#class label(random) ##we actualy don't this need label
            precl.append(cname)#predicted class name ##insert result into database
    
            print(temp_filename,predictions, iname, cname)#see result

            print(clothes_color)
            print(clothes_pattern)

            # 插入數據到數據庫（與你的程式碼保持一致）
            with open(filename, "rb") as f:
                img_o = f.read()
                img_no_bg = remove(img_o)
                
                with mysql.connector.connect(host="xxxx", user="xxxx", passwd="xxxx", database="ALL_CLOTHES_SQL", connection_timeout=6000) as db:
                    with db.cursor() as cursor:
                        sql = "INSERT INTO `MY_CLOTHES_TOP` (`FILENAME`, `CLOTHES_IMG`,`COLOR`,`PATTERN`, `TYPE`) VALUES (%s, %s, %s, %s, %s)"
                        img_name = uploaded_file.filename
                        values = (img_name, img_no_bg, clothes_color, clothes_pattern, cname)

                        cursor.execute(sql, values)
                        db.commit()
                        cursor.close()  

    if file_paths:
        upload_message = f'{len(file_paths)} 張圖片已成功上傳：{", ".join(file_paths)}'
        #return f'{len(file_paths)} 張圖片已成功上傳：{", ".join(file_paths)}'
    else:
        upload_message = '無效的文件格式或未選擇文件。'
        #return '無效的文件格式或未選擇文件。'
    return render_template('html_cnn_py.html', upload_message=upload_message)


def connect_to_database():
    return pymysql.connect(host='xxxx',
                           user='xxxx',
                           password='xxxx',
                           database='ALL_CLOTHES_SQL',
                           charset='utf8mb4',
                           cursorclass=pymysql.cursors.DictCursor)
    
def show_images(images):
    num_columns = max(len(images), 1)

    if num_columns == 1:
        fig, axs = plt.subplots(1, 1, figsize=(5, 5))
        axs.imshow(images[0])
        axs.axis('off')
    else:
        fig, axs = plt.subplots(1, num_columns, figsize=(15, 5))
        for i, image in enumerate(images):
            axs[i].imshow(image)
            axs[i].axis('off')

def generate_col_images(clothing_types):
    col2 = []
    col = []
    col_images = []

    for i in clothing_types:
        selected_data = get_random_clothing('MY_CLOTHES_TOP', i)

        if selected_data:
            col2.append(selected_data['FILENAME'])
            col.append(selected_data['COLOR'])

            image_data = selected_data['CLOTHES_IMG']
            image = Image.open(BytesIO(image_data))
            col_images.append(image)
            
    print("Length of col_images_re:", len(col_images))

    return col2, col, col_images

    # 從指定表格中隨機選取符合條件的衣物
def get_random_clothing(table_name, clothing_type ):
    conn = connect_to_database()
    cursor = conn.cursor()

    # 從指定表格中隨機選取一行
    query = f"SELECT `CLOTHES_IMG`,`FILENAME`,`PATTERN`,`COLOR`,`TYPE` FROM {table_name} WHERE TYPE = %s ORDER BY RAND() LIMIT 1"
    cursor.execute(query, (clothing_type))
    result = cursor.fetchone()
            
    # 關閉資料庫連接
    cursor.close()
    conn.close()
    return result

def check_colors(pairs):
    for i in pairs:
        pair_set = set(i)
        for forbidden_color, forbidden_list in forbidden.items():
            if pair_set.issubset(set(forbidden_list)):
                return 'not success'
    return 'success'

def get_pairs(col):
    pair = []
    for i in range(len(col)):
        for j in range(i + 1, len(col)):
            pair.append([col[i], col[j]])
    return pair


@app.route('/another_page')
def another_page():
    return render_template('CityDate.html')  # 導向到選擇城市和日期的HTML頁面


@app.route('/generate', methods=['POST'])
def generate_result():
    global avg_temp
    avg_temp = None
    city = request.form['city']
    year = request.form['year']
    month = request.form['month']
    day = str(request.form['day']).zfill(2)
    global location
    location = str(city)
    global dir_date
    dir_date = str(year+'/'+month+'/'+day)
    print(dir_date)

    result_temp = get_temperature(city, dir_date)
    img_buf_list = []
    if result_temp is not None:
        avg_temp = result_temp
        #衣服搭配公式 
        outfit_list1 = [
            "短袖 + 短褲" , 
            "短袖 + 長褲" ,
            "背心 + 短褲 ",
            "背心 + 長褲 ",
            "背心 + 長裙 ",
            "無洋 "
        ]
        outfit_list2 = [
            "短袖 + 短褲", 
            "短袖 + 長褲",  
            "短袖 + 長裙",    
            "背心 + 短褲 + 外套", 
            "無洋 + 外套",  
            "背心 + 長褲 + 外套",  
            "背心 + 長裙 + 外套"  
        ]
        outfit_list3 = [
            "長袖 + 短褲",
            "長袖 + 長褲",
            "長袖 + 長裙",
            "長洋",
            "高領 + 短褲",
            "帽踢 + 短褲",
            "短袖 + 短褲 + 外套",
            "短袖 + 長褲 + 外套",
            "短袖 + 長裙 + 外套"
        ]
        outfit_list4 = [
            "長袖 + 長褲",
            "長袖 + 長裙",
            "長袖 + 長褲 + 外套",
            "長袖 + 長裙 + 外套",
            "高領 + 長褲",
            "高領 + 長裙",
            "帽踢 + 長褲",
            "帽踢 + 長裙",
            "長洋 + 外套"
        ]                  
        outfit_list5 = [
            "長袖 + 長褲 + 外套",
            "長袖 + 長裙 + 外套",
            "帽踢 + 長褲 + 外套",
            "帽踢 + 長裙 + 外套",
            "高領 + 長褲 + 外套",
            "高領 + 長裙 + 外套",
            "帽踢 + 長褲 + 羽絨",
            "帽踢 + 長裙 + 羽絨",
            "高領 + 長褲 + 羽絨",
            "高領 + 長裙 + 羽絨",
            "長袖 + 長褲 + 羽絨",
            "長袖 + 長裙 + 羽絨",
            "長洋 + 羽絨"
        ]

        #def generate_outfit(temperature):
        all_lists = {
                'list1': outfit_list1,
                'list2': outfit_list2,
                'list3': outfit_list3,
                'list4': outfit_list4,
                'list5': outfit_list5
            }

        # 輸入溫度
        #temperature = avg_temp
        temperature = float(avg_temp) 

        if temperature >= 28:
            selected_list_key = 'list1'
        elif 25 <= temperature < 28:
            selected_list_key = 'list2'
        elif 21 <= temperature < 25:
            selected_list_key = 'list3'
        elif 16 <= temperature < 21:
            selected_list_key = 'list4'
        else:
            selected_list_key = 'list5' 
        

        selected_option = random.choice(all_lists[selected_list_key])
        print(selected_option)

        # 解析搭配，這裡簡單假設搭配是由 "T" 和 "B" 開頭的類型組成
        clothing_types = [item.strip() for item in selected_option.split('+')]

        col2 = []
        col = []
        col_images = []

        for i in clothing_types:
            selected_data = get_random_clothing('MY_CLOTHES_TOP', i)

            if selected_data:
                col2.append(selected_data['FILENAME'])
                col.append(selected_data['COLOR'])

                image_data = selected_data['CLOTHES_IMG']
                image = Image.open(BytesIO(image_data))
                col_images.append(image)

        # 检查 col_images 是否为空
        if len(col_images) == 0:
            selected_option = random.choice(all_lists[selected_list_key])
            # 重新生成图像
            clothing_types = [item.strip() for item in selected_option.split('+')]
            col2, col, col_images = generate_col_images(clothing_types)
            if len(col_images) == 0:
                selected_option = random.choice(all_lists[selected_list_key])
                # 重新生成图像
                clothing_types = [item.strip() for item in selected_option.split('+')]
                col2, col, col_images = generate_col_images(clothing_types)
                if len(col_images) == 0:
                    selected_option = random.choice(all_lists[selected_list_key])
                    # 重新生成图像
                    clothing_types = [item.strip() for item in selected_option.split('+')]
                    col2, col, col_images = generate_col_images(clothing_types)

        # 将圖片流返回到網頁
        
        fig, ax = plt.subplots()
        img_buf_list = []
        try:
            for img in col_images:
                img_buf = io.BytesIO()
                img.save(img_buf, format='PNG')
                img_buf.seek(0)
                img_base64 = base64.b64encode(img_buf.read()).decode('utf-8')
                img_buf_list.append(img_base64)
        except Exception as e:
            print(f"An error occurred: {e}")
        img_buf_list = img_buf_list if img_buf_list else []



    # 將圖片流返回到網頁
    
###     
        print("Length of col_images:", len(col_images))
        show_images(col_images)

        pairs = get_pairs(col)
        


        global forbidden
        forbidden = {
            'Red 紅色系': ['Red 紅色系', 'Red2 紅棕色系', '橘粉色系', 'Orange 橘色系', 'Yellow 黃色系', 'Green 綠色系', 'Purple 紫色系'],
            'Red2 紅棕色系': ['Red2 紅棕色系', 'Red 紅色系', '橘粉色系', 'Orange 橘色系', 'Green 綠色系', 'Purple 紫色系'],
            '橘粉色系': ['Red 紅色系', 'Red2 紅棕色系', '橘粉色系', 'Orange 橘色系', 'Green 綠色系'],
            'PINK 粉紅色系': ['Green 綠色系'],
            'Orange 橘色系': ['Red 紅色系', 'Red2 紅棕色系', '橘粉色系', 'Orange 橘色系', 'Green 綠色系'],
            'Yellow 黃色系': ['Red 紅色系', 'Red2 紅棕色系', '橘粉色系', 'Orange 橘色系', 'Green 綠色系', 'PINK 藕粉色系'],
            'Green 綠色系': ['Red 紅色系', 'Red2 紅棕色系', '橘粉色系', 'Orange 橘色系', 'PINK 藕粉色系'],
            'Cyan 青色系': ['Red 紅色系', 'Red2 紅棕色系'],
            'Purple 紫色系': ['Red 紅色系', 'Red2 紅棕色系', 'Orange 橘色系'],
            'PINK 藕粉色系': ['Red 紅色系', 'Red2 紅棕色系', 'Orange 橘色系', 'Green 綠色系'],
        }


        result_color=check_colors(pairs)


        # 保留原始溫度
        original_temperature = temperature

        # 如果結果是 'not success'，重新嘗試
        while result_color == 'not success':
            # 使用原始溫度重新生成搭配和顏色
            temperature = original_temperature
            selected_option = random.choice(all_lists[selected_list_key])
            clothing_types = [item.strip() for item in selected_option.split('+')]
            col_images2 = []
            col3 = []
            col2 = []
            col = []
            img_buf_list = []
            for i in clothing_types:
                selected_data = get_random_clothing('MY_CLOTHES_TOP', i)
                if selected_data:
                    col.append(selected_data['COLOR'])
                    col2.append(selected_data['FILENAME'])
                    col3.append(selected_data['TYPE'])
                    image_data = selected_data['CLOTHES_IMG']
                    image = Image.open(BytesIO(image_data))
                    col_images2.append(image)
                    fig, ax = plt.subplots()
                    try:
                        for img in col_images:
                            img_buf = io.BytesIO()
                            img.save(img_buf, format='PNG')
                            img_buf.seek(0)
                            img_base64 = base64.b64encode(img_buf.read()).decode('utf-8')
                            img_buf_list.append(img_base64) 
                    except Exception as e:
                        print(f"An error occurred: {e}")
                    # 在這裡將 img_buf_list 作為初始值定義
                    img_buf_list = img_buf_list if img_buf_list else []

                    

            print("Length of col_images2:", len(col_images2))
            show_images(col_images2)
            
            

            pairs = get_pairs(col)

            result = check_colors(pairs)
            print(result)
            print(col3)



            plt.show()
            #show_images(col_images)

        
        print("Final Filenames",col2)#final output filename


        return render_template('CityDate.html', city=city, year=year, month=month, day=day, avg_temp=avg_temp, img_buf_list=img_buf_list)
        
    else:
        return render_template('CityDate.html', city=city, year=year, month=month, day=day, error_message='無法查詢當日氣溫，請輸入「未來一週內」日期。')



###########        

@app.route('/all_clothes_page')  
def all_clothes_page():
    return render_template('get_clothes_all.html') #導向到全部衣服的HTML頁面

@app.route('/get_clothes_all')
def get_images():
    db = mysql.connector.connect(host="xxxx", user="xxxx", passwd="xxxx", database="ALL_CLOTHES_SQL",connection_timeout=6000)
    cursor = db.cursor()

    cursor.execute("SELECT `CLOTHES_IMG` FROM `MY_CLOTHES_TOP` ")  
    images = cursor.fetchall()

    db.close()

    encoded_images = [base64.b64encode(image[0]).decode('utf-8') for image in images]

    return jsonify(encoded_images)

@app.route('/contact_page')
def contact():
    return render_template('contact.html') 

if __name__ == '__main__':
    app.static_folder = 'static'
    app.run()
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/img/hero-bg.jpg HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/vendor/bootstrap-icons/bootstrap-icons.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/vendor/aos/aos.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/vendor/swiper/swiper-bundle.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/vendor/glightbox/css/glightbox.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/vendor/aos/aos.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:32:57] "GET /static/assets/vendor/purecounter/p


---------------------------------------
6.jpg
1/1 [==============================] - 0s 104ms/step


2023-12-05 14:33:26.058101: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


temp_image.png [[ 16.22078    11.077935   13.392809    3.4136958  27.298168   -4.1763515
   34.124866    3.323136   37.43325    -5.631608  -15.793776   -7.2625966]] 8 短袖
Red2 紅棕色系
y

---------------------------------------
8.jpg
1/1 [==============================] - 0s 17ms/step
temp_image.png [[ 24.26889    -8.425414   11.532331   10.419313   51.1633      5.8453846
   70.118866   23.662897  133.02423   -10.336085  -46.290474  -33.011185 ]] 8 短袖
Blue 藍色系
n

---------------------------------------
9.jpg
1/1 [==============================] - 0s 15ms/step
temp_image.png [[ 26.65759     6.2157884  24.022722   16.178629   22.367727  -13.043485
   55.49208     8.680949   72.829285   -4.3064065 -39.583122   -7.8109417]] 8 短袖
Brown 褐色系
y

---------------------------------------
10.jpg
1/1 [==============================] - 0s 14ms/step
temp_image.png [[ -3.266463    -4.8324428    3.984159    12.226343     4.9262094
  -10.814264     6.1730247    8.260852    16.040894    13.319085
   -0.658579

127.0.0.1 - - [05/Dec/2023 14:35:57] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/vendor/glightbox/css/glightbox.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/vendor/swiper/swiper-bundle.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/vendor/aos/aos.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/vendor/bootstrap-icons/bootstrap-icons.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/vendor/purecounter/purecounter_vanilla.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:35:57] "GET /static/assets/vendor/aos/aos.js HTTP/


---------------------------------------
0.jpg
1/1 [==============================] - 0s 21ms/step
temp_image.png [[ 14.379356    11.520352    35.88165     14.675337    26.088043
   -0.76693237  30.179504     7.505818    20.315973    29.450607
  -22.945751    -9.708214  ]] 2 外套
Brown 褐色系
y

---------------------------------------
1.jpg
1/1 [==============================] - 0s 17ms/step
temp_image.png [[ 24.032547   -1.3908343  22.258818   19.327913   20.688013    3.8100646
   43.75709    16.85494    78.10487     6.624042  -33.8382    -24.026169 ]] 8 短袖
Brown 褐色系
y

---------------------------------------
2.jpg
1/1 [==============================] - 0s 16ms/step
temp_image.png [[  4.849123    5.8639555  32.274406   -6.928475   33.767403    7.13416
    4.572051   -2.161331   19.979586   28.017756  -10.768513    5.674039 ]] 4 帽踢
Red 紅色系
y

---------------------------------------
3.jpg
1/1 [==============================] - 0s 16ms/step
temp_image.png [[ 28.092146   12.623845   54.874554 

127.0.0.1 - - [05/Dec/2023 14:44:27] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/vendor/bootstrap-icons/bootstrap-icons.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/vendor/glightbox/css/glightbox.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/vendor/aos/aos.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/vendor/swiper/swiper-bundle.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 14:44:27] "GET /static/assets/vendor/aos/aos.js HTT

2023/1/01


127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /upload_page HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/vendor/bootstrap-icons/bootstrap-icons.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/vendor/swiper/swiper-bundle.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/vendor/aos/aos.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/vendor/glightbox/css/glightbox.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/vendor/aos/aos.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2023 15:20:06] "GET /static/assets/vendor/purecounter/purecounter_vanilla.js H